In [1]:
import numpy as np
import sympy as sp
from sympy.utilities.lambdify import lambdify

# symbols
x, y, t = sp.symbols('x y t', real=True)
ax, ay, nu = sp.symbols('ax ay nu', real=True)

# manufactured solution (smooth, periodic-friendly)
u_star = sp.sin(2*x) * sp.cos(3*y) * sp.exp(-t)

# define your RHS operator F[u] symbolically
ux  = sp.diff(u_star, x)
uy  = sp.diff(u_star, y)
uxx = sp.diff(u_star, x, 2)
uyy = sp.diff(u_star, y, 2)

# Example: F[u] = - (a·∇u) + ν Δu   (right-hand-side of du/dt = F[u])
F_sym = -(ax*ux + ay*uy) + nu*(uxx + uyy)

# turn into fast NumPy-callable
F_fn  = lambdify((x, y, t, ax, ay, nu), sp.simplify(F_sym), 'numpy')
u_fn  = lambdify((x, y, t), u_star, 'numpy')


In [2]:
# YOU provide this. It should compute F_h[u] on a uniform grid.
# u: (Ny, Nx) array on cell centers, dx==dy==h, periodic shown below
def numeric_rhs(u, h, ax, ay, nu):
    # example: 2nd-order central Laplacian + 1st-order upwind advection (replace with yours)
    def roll(a, s, axis): return np.roll(a, s, axis=axis)
    # gradients (1st-order upwind for illustration)
    dux = np.where(ax >= 0, (u - roll(u, 1, 1))/h, (roll(u, -1, 1)-u)/h)
    duy = np.where(ay >= 0, (u - roll(u, 1, 0))/h, (roll(u, -1, 0)-u)/h)
    # Laplacian (2nd-order central)
    lap = (roll(u,1,1)+roll(u,-1,1)+roll(u,1,0)+roll(u,-1,0)-4*u)/h**2
    return -(ax*dux + ay*duy) + nu*lap


In [3]:
def eoc_from_errors(hs, errs):
    hs = np.array(hs, dtype=float); errs = np.array(errs, dtype=float)
    return np.log(errs[:-1]/errs[1:]) / np.log(hs[:-1]/hs[1:])

def l2_err(a, b, h):  # discrete L2 norm
    return np.sqrt(np.mean((a-b)**2))  # (scales the same way for EOC)

# parameters & test time
params = dict(ax=1.1, ay=-0.7, nu=0.05)
t0 = 0.3

sizes = [32, 64, 128, 256]   # refine by 2×
errs  = []; hs = []
for N in sizes:
    # domain [0, 2π]×[0, 2π] (periodic example)
    Lx = Ly = 2*np.pi
    Nx = Ny = N
    xs = np.linspace(0, Lx, Nx, endpoint=False)
    ys = np.linspace(0, Ly, Ny, endpoint=False)
    X, Y = np.meshgrid(xs, ys, indexing='xy')
    h = Lx/Nx  # = Ly/Ny

    u_exact = u_fn(X, Y, t0)
    F_exact = F_fn(X, Y, t0, params['ax'], params['ay'], params['nu'])
    F_num   = numeric_rhs(u_exact, h, **params)

    errs.append(l2_err(F_num, F_exact, h))
    hs.append(h)

eoc = eoc_from_errors(hs, errs)
print("h:", hs)
print("||F_h[u*]-F[u*]||:", errs)
print("EOC (between successive grids):", eoc)


h: [0.19634954084936207, 0.09817477042468103, 0.04908738521234052, 0.02454369260617026]
||F_h[u*]-F[u*]||: [np.float64(0.3779685940501155), np.float64(0.19242750274093334), np.float64(0.09682945645858029), np.float64(0.048537029788524616)]
EOC (between successive grids): [0.97395135 0.99079711 0.99636017]
